이전에 영화 리뷰를 이용하여 172 개의 영화의 document vectors 를 학습했습니다. 이 벡터를 이용하면 리뷰가 비슷한 영화들을 군집화 할 수 있습니다.

In [1]:
import config
import gensim
from navermovie_comments import load_trained_embedding
from navermovie_comments import load_id_to_movie
import warnings

warnings.filterwarnings('ignore')
print('Gensim version = {}'.format(gensim.__version__))

soynlp=0.0.491
added lovit_textmining_dataset
Gensim version = 3.6.0


Doc2Vec에서 document (영화)의 임베딩 차원은 300이며, 영화의 개수는 172개임을 확인할 수 있습니다. 

In [2]:
tag_to_movie = load_id_to_movie()
tag_to_movie = {'#%s'%idx:movie for idx, movie in tag_to_movie.items()}

doc2vec_model = load_trained_embedding(
    data_name='large',
    tokenize='soynlp_unsup',
    embedding='doc2vec')

doctags = doc2vec_model.docvecs.doctags

print('docvec shape = {}'.format(doc2vec_model.docvecs.vectors_docs.shape))

docvec shape = (172, 100)


각 row 에 해당하는 영화 이름도 idx_to_movie 에 저장합니다.

In [3]:
tag_to_idx = {tag:info.offset for tag, info in doctags.items()}
idx_to_tag = [tag for tag, idx in sorted(tag_to_idx.items(), key=lambda x:x[1])]
idx_to_movie = [tag_to_movie[tag] for tag in idx_to_tag]

print('example of idx_to_movie = {}'.format(idx_to_movie[:3]))

example of idx_to_movie = ['고사 두 번째 이야기: 교생실습', '박쥐', '해무']


Scikit-learn 의 k-means 를 이용하여 학습합니다.
    
```python
class KMeans(n_clusters=8, init='k-means++', n_init=10,
    max_iter=300, tol=0.0001, precompute_distances='auto',
    verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm='auto'):

    # ...
```

| Parameter | Help |
| --- | --- |
| n_clusters | k. 군집의 개수는 예상하는 것보다 크게 설정 |
| n_init | 반복 횟수는 1로 설정 |
| max_iter | 큰 데이터라 하더라도 10 ~ 20 번 안에 수렴하기 때문에 작게 설정 |
| verbose | True 설정하면 iteration 마다 progress 가 출력 |

Scikit-learn 은 `metrics='cosine'` 을 지원하지 않습니다. Scikit-learn 을 이용하여 비슷한 학습 결과를 얻고 싶을 때에는 반드시 row normalize 을 하시기 바랍니다.

In [4]:
from sklearn.preprocessing import normalize

movie_vectors = doc2vec_model.docvecs.vectors_docs_norm
movie_vectors = normalize(movie_vectors)

kmeans 는 .fit_predict() 와 .fit_transform() 을 제공합니다. predict 함수는 각 row 의 cluster label 을 return 합니다.

In [5]:
%%time

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, max_iter=10, n_init=1, verbose=0)
clusters = kmeans.fit_predict(movie_vectors)

CPU times: user 104 ms, sys: 176 ms, total: 280 ms
Wall time: 112 ms


In [6]:
clusters

array([7, 0, 0, 2, 8, 2, 5, 0, 4, 7, 7, 5, 1, 5, 1, 0, 0, 1, 8, 1, 6, 2,
       3, 8, 9, 3, 0, 0, 9, 6, 1, 8, 8, 3, 6, 3, 2, 9, 2, 3, 0, 0, 0, 3,
       1, 0, 2, 1, 4, 9, 8, 9, 8, 5, 3, 1, 0, 2, 1, 6, 7, 0, 2, 8, 0, 8,
       1, 9, 6, 9, 4, 0, 9, 9, 7, 8, 3, 1, 1, 1, 2, 3, 1, 7, 3, 8, 4, 5,
       8, 7, 8, 2, 3, 8, 1, 5, 1, 6, 1, 2, 1, 9, 1, 7, 2, 8, 2, 1, 6, 4,
       0, 3, 7, 2, 8, 1, 0, 6, 6, 4, 6, 8, 8, 8, 6, 1, 6, 1, 3, 0, 7, 8,
       8, 2, 1, 5, 1, 6, 4, 9, 6, 2, 5, 0, 9, 6, 5, 8, 2, 3, 5, 1, 6, 3,
       7, 1, 7, 2, 8, 0, 4, 8, 1, 2, 1, 1, 1, 9, 8, 3, 1, 8], dtype=int32)

fit_transform 은 각 row 와 k 개의 centroid 와의 거리를 출력합니다.

In [7]:
distance = kmeans.fit_transform(movie_vectors)
distance.shape

(172, 10)

같은 cluster label 을 지니는 영화 별로 그룹을 만듭니다.

In [8]:
import numpy as np

unique_labels = np.unique(clusters)
for label in unique_labels:
    idxs = np.where(clusters == label)[0]
    print('cluster #{} : {} movies'.format(label, idxs.shape[0]))

cluster #0 : 20 movies
cluster #1 : 32 movies
cluster #2 : 19 movies
cluster #3 : 16 movies
cluster #4 : 8 movies
cluster #5 : 10 movies
cluster #6 : 16 movies
cluster #7 : 12 movies
cluster #8 : 26 movies
cluster #9 : 13 movies


각 cluster 별로 10개씩 영화의 이름을 출력하여, 군집화 결과가 어떤 느낌인지 확인해봅시다. 앞서 만들어둔 row2movie를 이용합니다. 

In [9]:
for label in unique_labels:
    idxs = np.where(clusters == label)[0]
    print('\ncluster #{} : {} movies'.format(label, idxs.shape[0]))
    for idx in idxs:        
        print('  > {}'.format(idx_to_movie[idx]))


cluster #0 : 20 movies
  > 박쥐
  > 해무
  > 베를린
  > 아가씨
  > 내가 살인범이다
  > 신세계
  > 곡성(哭聲)
  > 검은 사제들
  > 악마를 보았다
  > 좋은 놈, 나쁜 놈, 이상한 놈
  > 용의자
  > 관상
  > 최종병기 활
  > 감시자들
  > 역린
  > 아저씨
  > 아수라
  > 테이큰
  > 추격자
  > 세븐 데이즈

cluster #1 : 32 movies
  > 다크 나이트 라이즈
  > 배트맨 대 슈퍼맨: 저스티스의 시작
  > 메이즈 러너: 스코치 트라이얼
  > 캡틴 아메리카: 시빌 워
  > 빅 히어로
  > 인디펜던스 데이: 리써전스
  > 제이슨 본
  > 쥬라기 월드
  > 엑스맨: 데이즈 오브 퓨처 패스트
  > 워크래프트: 전쟁의 서막
  > 엣지 오브 투모로우
  > 맨 오브 스틸
  > 킹스맨 : 시크릿 에이전트
  > 미션 임파서블: 로그네이션
  > 미스 페레그린과 이상한 아이들의 집
  > 신비한 동물사전
  > 혹성탈출: 반격의 서막
  > 아이언맨 3
  > 어벤져스: 에이지 오브 울트론
  > 데드풀
  > 터미네이터 제니시스
  > 아이언맨
  > 수어사이드 스쿼드
  > 닥터 스트레인지
  > 어벤져스
  > 엑스맨: 아포칼립스
  > 트랜스포머: 사라진 시대
  > 캡틴 아메리카: 윈터 솔져
  > 매드맥스: 분노의 도로
  > 나우 유 씨 미 2
  > 어메이징 스파이더맨 2
  > 마션

cluster #2 : 19 movies
  > 해운대
  > 국가대표
  > 미녀는 괴로워
  > 포화 속으로
  > 의형제
  > 광해, 왕이 된 남자
  > 해바라기
  > 고지전
  > 왕의 남자
  > 괴물
  > 화려한 휴가
  > 인생은 아름다워
  > 웰컴 투 동막골
  > 투사부일체
  > 우리들의 행복한 시간
  > 다세포 소녀
  > 쌍화점
  > 라디오 스타
  > 라스트 갓파더

cluster #3 : 16 movies
  > 터널
  > 대호

kmeans 의 `KMeans.cluster_centers_` 는 각 centroid 좌표입니다. 

Doc2Vec 임베딩 벡터의 크기가 100 이고, 클러스터의 개수가 10이기 때문에 centers 행렬의 크기는 (10, 100)입니다. 

In [10]:
kmeans.cluster_centers_.shape

(10, 100)